In [1]:
import BMTextTokenizer as bmt
import pandas as pd
import networkx as nx
import time
import matplotlib.pyplot as plt
import stanza
from itertools import permutations
%matplotlib inline

2022-02-02 10:47:04 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | craft      |
| pos       | craft      |
| lemma     | craft      |
| depparse  | craft      |
| ner       | bionlp13cg |

2022-02-02 10:47:04 INFO: Use device: cpu
2022-02-02 10:47:04 INFO: Loading: tokenize
2022-02-02 10:47:04 INFO: Loading: pos
2022-02-02 10:47:04 INFO: Loading: lemma
2022-02-02 10:47:04 INFO: Loading: depparse
2022-02-02 10:47:04 INFO: Loading: ner
2022-02-02 10:47:05 INFO: Done loading processors!
2022-02-02 10:47:05 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2022-02-02 10:47:05 INFO: Use device: cpu
2022-02-02 10:47:05 INFO: Loading: tokenize
2022-02-02 10:47:05 INFO: Loading: pos
2022-02-02 10:47:05 INFO: Loading: lemma
2022-02-02 10:47:05 INFO: Loading: depparse
2022-

In [2]:
# Default stanza pipeline to be used throughout this notebook
nlp = stanza.Pipeline('en', package='craft', processors={'ner': 'BioNLP13CG'})
pretokenized_nlp = stanza.Pipeline(lang='en', package='craft', processor='tokenize', tokenize_pretokenized=True) 

2022-02-02 10:47:06 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | craft      |
| pos       | craft      |
| lemma     | craft      |
| depparse  | craft      |
| ner       | bionlp13cg |

2022-02-02 10:47:06 INFO: Use device: cpu
2022-02-02 10:47:06 INFO: Loading: tokenize
2022-02-02 10:47:06 INFO: Loading: pos
2022-02-02 10:47:06 INFO: Loading: lemma
2022-02-02 10:47:06 INFO: Loading: depparse
2022-02-02 10:47:07 INFO: Loading: ner
2022-02-02 10:47:07 INFO: Done loading processors!
2022-02-02 10:47:07 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2022-02-02 10:47:07 INFO: Use device: cpu
2022-02-02 10:47:07 INFO: Loading: tokenize
2022-02-02 10:47:07 INFO: Loading: pos
2022-02-02 10:47:07 INFO: Loading: lemma
2022-02-02 10:47:07 INFO: Loading: depparse
2022-

In [3]:
path = '/Volumes/GoogleDrive/My Drive/_My Data Analytics Exercise/Exercise/Biopython/CE/'

In [4]:
%%time
df = pd.read_csv(path+'all_interactions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  1124 non-null   object
 1   sentence       1124 non-null   object
 2   entities       1124 non-null   object
 3   interaction    1124 non-null   object
 4   relationship   1124 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 44.0+ KB
CPU times: user 20.3 ms, sys: 7.75 ms, total: 28 ms
Wall time: 46 ms


In [5]:
%%time
pos_tag = []
with open('upos_tag.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        pos_tag.append(line.replace('\n',''))
pos_tag       

CPU times: user 1.74 ms, sys: 3.19 ms, total: 4.93 ms
Wall time: 11.6 ms


['VERB',
 'NOUN',
 'ADJ',
 'NUM',
 'ADP',
 'PUNCT',
 'CONJ',
 'DET',
 'PROPN',
 'PRON',
 'SCONJ',
 'SYM',
 'ADV',
 'AUX',
 'PART',
 'X',
 'CCONJ']

In [6]:
with_interactions = df[df['relationship']==1].copy()
without_interactions = df[df['relationship']==0].copy()

In [7]:
with_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 1 to 1122
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  825 non-null    object
 1   sentence       825 non-null    object
 2   entities       825 non-null    object
 3   interaction    825 non-null    object
 4   relationship   825 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 38.7+ KB


In [8]:
without_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 1123
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  299 non-null    object
 1   sentence       299 non-null    object
 2   entities       299 non-null    object
 3   interaction    299 non-null    object
 4   relationship   299 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 14.0+ KB


In [9]:
with_interactions.head()

,interactionId,sentence,entities,interaction,relationship
1,BioInfer_d682,The recombinant material is similar to authent...,"[('skeletal actin#', 'T1', 'Individual_protein...","{'R1': ('T1', 'T3')}",1
3,BioInfer_d595,The common denominator is impaired beta-cateni...,"[('beta-catenin#', 'T1', 'Individual_protein',...","{'R1': ('T1', 'T2'), 'R2': ('T1', 'T3')}",1
8,BioInfer_d538,"Significantly, those actin mutants exhibiting ...","[('actin#', 'T4', 'Gene/protein/RNA', (21, 26)...","{'R1': ('T1', 'T2'), 'R2': ('T1', 'T3')}",1
11,BioInfer_d54,Analyses of dynamic light scattering data by s...,"[('actin#', 'T6', 'Individual_protein', (129, ...","{'R1': ('T1', 'T3'), 'R2': ('T2', 'T4'), 'R3':...",1
12,BioInfer_d777,Treatment with HNE resulted in activation of e...,[('extracellular signal-regulated protein kina...,"{'R1': ('T1', 'T6'), 'R2': ('T2', 'T6')}",1


### Preprocessing of Sentences for further analysis

In [17]:
%%time
# Those sentences that have 'relationship' as indicated in original data set

directory = '/Volumes/GoogleDrive/My Drive/_My Data Analytics Exercise/Exercise/Biopython/CE/graph/'
from tqdm import tqdm
from random import sample
tokens_lst = dict()
index = sample(sorted(with_interactions.index),100)
special_attention = []
for i in tqdm(index):
    ids = with_interactions.loc[i, 'interactionId']
    sent = bmt.cleanSentencefmDF(with_interactions.loc[i, 'sentence'])
    print (i, sent)
    ents = with_interactions.loc[i, 'entities']
    doc = bmt.dynamicTokenizer(pretokenized_nlp, sent) # retokenize sentence to group chunks into single meaningful biomedical entity
    entities = bmt.extractEntities(bmt.extractEntitiesfmDF(ents))
    #print ('Number of unique entity:', len(entities), entities)
    H = bmt.createTokenGraphwithContraction(doc) # create an undirected graph connecting various tokens
    filename = directory+ids+".gpickle"
    nx.write_gpickle(H, filename)
    tokens = bmt.extractTokensOfInterestfmGraph(H, entities, threshold=0.8)
    #print ('Number of Tokens:', len(tokens), tokens)
    tokens_lst[ids] = tokens

  0%|                                                   | 0/100 [00:00<?, ?it/s]

189 Studies on cultured cells have suggested that both alpha-catenin and plakoglobin are important for the adhesive function of cadherins. #
Prev size: 21 Vs. Next size 17


  1%|▍                                          | 1/100 [00:00<00:30,  3.25it/s]

Prev size: 17 Vs. Next size 17
353 These results suggest that profilin may be involved in the pathogenesis of glomerulonephritis by reorganizing actin cytoskeleton. #
Prev size: 19 Vs. Next size 18


  2%|▊                                          | 2/100 [00:00<00:35,  2.76it/s]

...

Prev size: 35 Vs. Next size 35
323 Although neither activation of BCK1 (MEKK) by the dominant BCK1-20 mutation nor increased dosage of MKK1 (MEK) or MPK1 (MAP kinase) mimicked PKC1 as a glc7-10 dosage suppressor, extra copies of genes encoding upstream components of the Pkc1p pathway such as ROM2, RHO2, HCS77/WSC1/SLG1 and MID2 also suppressed glc7-10 effectively. #
Prev size: 55 Vs. Next size 44
Prev size: 44 Vs. Next size 39
Prev size: 39 Vs. Next size 35


 99%|█████████████████████████████████████████▌| 99/100 [01:48<00:01,  1.15s/it]

Prev size: 35 Vs. Next size 35
950 Analyses of hormone regulation of expression of mRNA for the aleurone RNase revealed that, like the pattern for alpha-amylase, mRNA levels increased in the presence of gibberellic acid, and its antagonist abscisic acid prevented this effect Quantitative studies at early times demonstrated that cycloheximide treatment of aleurone layers increased mRNA levels 4-fold, whereas a combination of gibberellin plus cycloheximide treatment was required to increase alpha-amylase mRNA levels to the same extent. #
Prev size: 77 Vs. Next size 63
Prev size: 63 Vs. Next size 60
Prev size: 60 Vs. Next size 59


100%|█████████████████████████████████████████| 100/100 [01:49<00:00,  1.10s/it]

Prev size: 59 Vs. Next size 59
CPU times: user 13min 6s, sys: 2min 43s, total: 15min 49s
Wall time: 1min 49s
